# IMPORTS & MODULES

In [1]:
import urllib
import bs4
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib
import bs4
from bs4 import BeautifulSoup
from urllib import request
import pandas as pd
import re

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("enable-automation")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")
# Specify the path to the WebDriver (update this with your WebDriver path)
driver_path = '/home/website_football/chromedriver-linux64/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# SCRAPP TRANSFERMAKT

In [3]:
transfert_markt = "https://www.transfermarkt.fr/wettbewerbe/europa"
driver.get(transfert_markt)
html = driver.page_source
print(driver.title)

Championnats et coupes européens | Transfermarkt


In [4]:
nom_championnat =driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td/table/tbody/tr/td[2]//a')
url_championnat =driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td/table/tbody/tr/td[2]//a')
nombre_club = driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td[3]')

leagues =[]

for i in range(0,5):
    temporary={'nom_championnat':nom_championnat[i].get_attribute('title'),
                           'url_championnat':url_championnat[i].get_attribute('href'),
                           'nombre_club':int(nombre_club[i].get_attribute('innerText'))     }
    leagues.append(temporary)

#driver.quit()
df_leagues = pd.DataFrame(leagues)
df_leagues.head()

,nom_championnat,url_championnat,nombre_club
0,Premier League,https://www.transfermarkt.fr/premier-league/st...,20
1,LaLiga,https://www.transfermarkt.fr/laliga/startseite...,20
2,Serie A,https://www.transfermarkt.fr/serie-a/startseit...,20
3,Bundesliga,https://www.transfermarkt.fr/bundesliga/starts...,18
4,Ligue 1,https://www.transfermarkt.fr/ligue-1/startseit...,18


In [5]:
clubs =[]
for championnat in range(0,5):
        league = df_leagues['url_championnat'][championnat]
        driver.get(league)
        nom_championnat = df_leagues['nom_championnat'][championnat]
        
        nom_club =driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td[3]/a[1]')
        url_club =driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td[2]/a[1]')
        nombre_joueur = driver.find_elements('xpath','//*[@id="yw1"]/table/tbody/tr/td[3]/a[1]')
        
        
        stop_range=df_leagues['nombre_club'][championnat]
        
        for i in range(0,stop_range):
            temporary={'nom_championnat':nom_championnat,
                'nom_club':nom_club[i].get_attribute('title'),
                           'url_club':url_club[i].get_attribute('href'),
                           'nombre_joueur':int(nombre_joueur[i].get_attribute('innerText'))     }
            clubs.append(temporary)
            
df_clubs = pd.DataFrame(clubs)
df_clubs.head(100)

,nom_championnat,nom_club,url_club,nombre_joueur
0,Premier League,Manchester City,https://www.transfermarkt.fr/manchester-city/s...,29
1,Premier League,Arsenal FC,https://www.transfermarkt.fr/fc-arsenal/starts...,24
2,Premier League,FC Liverpool,https://www.transfermarkt.fr/fc-liverpool/star...,25
3,Premier League,Chelsea FC,https://www.transfermarkt.fr/fc-chelsea/starts...,31
4,Premier League,Tottenham Hotspur,https://www.transfermarkt.fr/tottenham-hotspur...,30
...,...,...,...,...
91,Ligue 1,Le Havre AC,https://www.transfermarkt.fr/ac-le-havre/start...,27
92,Ligue 1,AJ Auxerre,https://www.transfermarkt.fr/aj-auxerre/starts...,29
93,Ligue 1,Montpellier Hérault SC,https://www.transfermarkt.fr/montpellier-hsc/s...,25
94,Ligue 1,AS Saint-Étienne,https://www.transfermarkt.fr/as-saint-etienne/...,27


In [8]:
joueurs =[]

for club in range(0,len(df_clubs)):
    equipe = df_clubs['url_club'][club]
    driver.get(equipe)
    nom_championnat = df_clubs['nom_championnat'][club]
    nom_club = df_clubs['nom_club'][club]
    
    infos_joueurs = driver.find_elements('xpath','//*[@id="yw1"]//td[@class="hauptlink"]/a')
    vm = driver.find_elements('xpath','//*[@id="yw1"]//td[@class="rechts hauptlink"]/a')
    i = 0
    
    #stop_range=df_clubs['nombre_joueur'][club]

    for infos_joueur in infos_joueurs:
        name_raw_html = infos_joueur.get_attribute("innerHTML") 
        name_raw = BeautifulSoup(name_raw_html, 'html.parser')
        player_name = name_raw.get_text().strip()

        valeur_marchande = vm[i].get_attribute('innerText')

        temporary={
            'nom_championnat':nom_championnat,
            'nom_club':nom_club,
            'nom_joueur':player_name,
            'valeur_marchande':valeur_marchande,
            'url_joueur':infos_joueur.get_attribute('href')
        }
        joueurs.append(temporary)
df_joueurs = pd.DataFrame(joueurs)

In [9]:
#df = pd.DataFrame(joueurs)
df_joueurs.to_csv('transfermkt_url.csv')

In [9]:
def extract_player_info(text):
    # Extract age (number inside parentheses after "Naissance (âge):")
    age_match = re.search(r"Naissance \(âge\):\n.*\((\d+)\)", text)
    age = age_match.group(1) if age_match else None

    # Extract birthdate (before the parentheses in "Naissance (âge):")
    birthdate_match = re.search(r"Naissance \(âge\):\n([\d\w. ]+)", text)
    birthdate = birthdate_match.group(1).strip() if birthdate_match else None

    # Extract nationality (handle multiple nationalities)
    #nationality_match = re.findall(r"Nationalité:\n(?:\s+\xa0\xa0)?([\w\s]+)\nPosition", text)
    nationality_match = re.findall(r"Nationalité:\n(?:\s+\xa0\xa0)?([\w\sÀ-ÿ'’-]+)\nPosition:", text)
    nationality = "-".join(n.strip() for n in nationality_match if n.strip()) if nationality_match else None
    if (nationality != None):
        nationality = nationality.split("\n")[0]
    
    # Extract contract end date
    contract_match = re.search(r"Contrat jusqu’à:\n([\d\w ]+)", text)
    contract_until = contract_match.group(1).strip() if contract_match else None

    # Extract position
    position_match = re.search(r"Position:\n([^\n]+)", text)
    position = position_match.group(1).strip() if position_match else None

    # Extract dominant foot
    foot_match = re.search(r"Pied:\n(\w+)", text)
    foot = foot_match.group(1).strip() if foot_match else None

    # Extract height (convert to float for numerical operations)
    height_match = re.search(r"Taille:\n([\d,.]+)\s?m", text)
    height = float(height_match.group(1).replace(",", ".")) if height_match else None

    # Store extracted data in a dictionary
    player_info = {
        "Age": age,
        "Birthdate": birthdate,
        "Nationality": nationality,
        "Contract Until": contract_until,
        "Position": position,
        "Foot": foot,
        "Height": height,
    }

    return player_info

In [255]:
data =[]
for player in range(0,len(df)):
    joueur = df['url_joueur'][player]
    driver.get(joueur)
    infos = {}
    
    try:
        time.sleep(2)
        element = driver.find_elements('xpath','/html/body/div/main/div[2]/div[1]/div[2]/div/div[2]/div')
        infos = extract_player_info(element[0].get_attribute('innerText'))
        infos['nom_championnat'] = df['nom_championnat'][player]
        infos['nom_club'] = df['nom_club'][player]
        infos['nom_joueur'] = df['nom_joueur'][player]
        data.append(infos)
    except Exception as e:
        print("here : ",player)
        print(e)
        pass
dataframe =  pd.DataFrame(data)

here :  61
list index out of range
here :  63
list index out of range
here :  68
list index out of range
here :  152
list index out of range
here :  171
list index out of range
here :  177
list index out of range
here :  180
list index out of range
here :  195
list index out of range


In [15]:
#df.head(2)
dataframe.tail(2)

""


# SCRAPP FBREF

In [345]:
driver.get('https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats')
squad_rows = driver.find_elements('xpath','//table[contains(@class, "stats_table")]/tbody/tr/td[@data-stat="team"]') 
  #'//table[@id="big5_table"]//tbody//tr//th[@data-stat="team"]')

# Extract squad names and URLs
squads = {}
for row in squad_rows:
    squad_name = row.text  # Extract squad name
    if (squad_name != 'Squad'):
        link_element = row.find_element(By.TAG_NAME, "a")  # Find the anchor tag
        squad_url = link_element.get_attribute("href") if link_element else None 
        squads[squad_name] = squad_url

In [411]:
squads_temp = {key: squads[key] for key in list(squads.keys())[:2]}

In [452]:
# Extract players names and URLs
player_data = []
for squad, squad_url in squads.items():
    try:
        time.sleep(2)
        driver.get(squad_url)
        positions = driver.find_elements('xpath','//table[contains(@class, "stats_table")]/tbody/tr/td[@data-stat="position"]')  
        players = driver.find_elements('xpath','//table[contains(@class, "stats_table")]//tbody//tr//th[@data-stat="player"]')

        for i in range(30): # TO_CHANGE don't understand why but got +200 players by teams (maybe due to contains in : table[contains(@class, "stats_table")] )
            name = players[i].text.strip()
            position = positions[i].text.split(',')[0].strip()
            link_element = players[i].find_element(By.TAG_NAME, "a")
            url = link_element.get_attribute("href") if link_element else None
            player_data.append({"team":squad, "name": name, "position":position, "player_url": url})

    except Exception as e:
        print("here : ",player)
        print(e)
        pass

In [457]:
df_fbref = pd.DataFrame(player_data)
df_fbref.drop_duplicates()
df_fbref.to_csv('data_df_fbref_url.csv')

,team,name,position,player_url
0,Bayern Munich,Joshua Kimmich,MF,https://fbref.com/en/players/49296448/Joshua-K...
1,Bayern Munich,Kim Min-jae,DF,https://fbref.com/en/players/e0f8151c/Kim-Min-jae
2,Bayern Munich,Harry Kane,FW,https://fbref.com/en/players/21a66f6a/Harry-Kane
3,Bayern Munich,Dayot Upamecano,DF,https://fbref.com/en/players/d248cd8f/Dayot-Up...
4,Bayern Munich,Manuel Neuer,GK,https://fbref.com/en/players/8778c910/Manuel-N...


In [462]:
df_fbref.head(20)

,team,name,position,player_url
0,Bayern Munich,Joshua Kimmich,MF,https://fbref.com/en/players/49296448/Joshua-K...
1,Bayern Munich,Kim Min-jae,DF,https://fbref.com/en/players/e0f8151c/Kim-Min-jae
2,Bayern Munich,Harry Kane,FW,https://fbref.com/en/players/21a66f6a/Harry-Kane
3,Bayern Munich,Dayot Upamecano,DF,https://fbref.com/en/players/d248cd8f/Dayot-Up...
4,Bayern Munich,Manuel Neuer,GK,https://fbref.com/en/players/8778c910/Manuel-N...
5,Bayern Munich,Alphonso Davies,DF,https://fbref.com/en/players/d781d855/Alphonso...
6,Bayern Munich,Michael Olise,FW,https://fbref.com/en/players/c4486bac/Michael-...
7,Bayern Munich,Jamal Musiala,MF,https://fbref.com/en/players/2c0558b8/Jamal-Mu...
8,Bayern Munich,Raphaël Guerreiro,DF,https://fbref.com/en/players/048a1cbc/Raphael-...
9,Bayern Munich,Aleksandar Pavlovic,MF,https://fbref.com/en/players/2658c82f/Aleksand...


In [544]:
def parse_scouting_report(report, name):
    try:
        lines = report[0].text.split("Player compared")[0].split("\n")
        # Extract position (first line containing "vs.")
        position = None
        for line in lines:
            if line.startswith("vs."):
                position = line.replace("vs.", "").strip()
                break
        
            # Find the starting index of stats table
        start_idx = None
        for i, line in enumerate(lines):
            if "Statistic Per 90 Percentile" in line:
                start_idx = i + 1
                break

        # Extract stats
        stats = {}
        for line in lines[start_idx:]:
            parts = line.strip().rsplit(" ", 2)  # Split from the right
            if len(parts) != 3:
                continue  # Skip invalid lines

            stat_name = parts[0].strip()
            per_90 = parts[1].strip()
            percentile = parts[2].strip()

            # Handle percentage values
            if "%" in per_90:
                per_90 = per_90.replace("%", "")
            
            stats[f"{stat_name} Per 90"] = per_90
            stats[f"{stat_name} Percentile"] = percentile

        # Create structured player data
        player_data = {
            "Name": name,
            "Position": position
        }
        player_data.update(stats)
        return player_data  
    except Exception as e:
        print(name)
        print(e)
        return None

In [560]:
#//*[@id="scout_summary_AM"]

# Lists to store data for each position
data_MF = []
data_FW = []
data_CB = []
data_FB = []
data_GK = []
data_AM = []  

for player in range(0,len(df_fbref)):
    time.sleep(2)
    joueur = df_fbref['player_url'][player]
    name = df_fbref['name'][player]
    driver.get(joueur)
    table_stats = driver.find_elements('xpath','//*[@id="all_scout_summary"]')
    player_data = parse_scouting_report(table_stats, name)
    if player_data:
        print(player)
        position = player_data["Position"]
        
        # Add to the corresponding dataset
        if "Midfielders" in position:
            data_MF.append(player_data)
        elif "Forwards" in position:
            data_FW.append(player_data)
        elif "Center Backs" in position:
            data_CB.append(player_data)
        elif "Fullbacks" in position:
            data_FB.append(player_data)
        elif "Goalkeepers" in position:
            data_GK.append(player_data)
        elif "Att Mid / Wingers" in position:
            data_AM.append(player_data)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Eric Dier
list index out of range
João Palhinha
list index out of range
Sacha Boey
list index out of range
Daniel Peretz
list index out of range
Mathys Tel
list index out of range
Sven Ulreich
list index out of range
Josip Stanišić
list index out of range
Adam Aznou
list index out of range
Arijon Ibrahimović
list index out of range
Jonah Kusi-Asare
list index out of range
Noel Nkili
list index out of range
Anthony Pavlesic
list index out of range
Max Schmitt
list index out of range
Jonas Urbig
list index out of range
Bradley Barcola
list index out of range
Achraf Hakimi
list index out of range
Gianluigi Donnarumma
list index out of range
Willian Pacho
list index out of range
Warren Zaïre-Emery
list index out of range
Vitinha
list index out of range
Lee Kang-in
list index out of range
João Neves
list index out of range
Ousmane Dembélé
list index out of range
Fabián Ruiz Peña
list index out of range
Marquinhos
list index out of range
Lucas Beraldo
li

KeyboardInterrupt: 

In [526]:
df_AM = pd.DataFrame(data_AM)
df_CB = pd.DataFrame(data_CB)
#data_AM

In [527]:
df_CB

,Name,Position,Non-Penalty Goals Per 90,Non-Penalty Goals Percentile,npxG: Non-Penalty xG Per 90,npxG: Non-Penalty xG Percentile,Shots Total Per 90,Shots Total Percentile,Assists Per 90,Assists Percentile,...,Tackles Per 90,Tackles Percentile,Interceptions Per 90,Interceptions Percentile,Blocks Per 90,Blocks Percentile,Clearances Per 90,Clearances Percentile,Aerials Won Per 90,Aerials Won Percentile
0,Kim Min-jae,Center Backs,0.09,80,0.06,73,0.40,33,0.03,59,...,1.96,83,1.76,94,1.11,32,2.78,8,3.10,88
1,Dayot Upamecano,Center Backs,0.07,75,0.06,75,0.61,67,0.00,27,...,1.78,74,1.55,86,1.18,39,2.12,1,2.73,80
2,Eric Dier,Center Backs,0.00,18,0.06,69,0.85,88,0.05,78,...,1.15,22,0.85,23,1.50,72,3.39,26,2.54,76
3,Willian Pacho,Center Backs,0.00,18,0.02,10,0.08,1,0.08,93,...,1.87,78,1.61,88,1.53,73,3.98,43,1.61,25


In [567]:
print(df_fbref['player_url'][50])
joueur = df_fbref['player_url'][50]
name = df_fbref['name'][50]
driver.get(joueur)
print(driver.current_url)
table_stats = driver.find_elements('xpath','//*[@id="all_scout_summary"]') 
print(table_stats)
print(name)
player_data = parse_scouting_report(table_stats, name)

https://fbref.com/en/players/c0b3669f/Randal-Kolo-Muani
https://fbref.com/en/players/c0b3669f/Randal-Kolo-Muani
[]
Randal Kolo Muani
Randal Kolo Muani
list index out of range


In [549]:
driver.find_elements('xpath','//*[@id="all_scout_summary"]')

[]

In [565]:
driver.close()

In [561]:
joueur = df_fbref['player_url'][5]
driver.get(joueur)
#print(driver.page_source)